<a href="https://colab.research.google.com/github/nnilayy/MedGPT/blob/main/llama_cpp_webserver_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%bash
pip install llama-cpp-python

In [ ]:
from llama_cpp import Llama

llm = Llama.from_pretrained(
    repo_id="Qwen/Qwen2-0.5B-Instruct-GGUF",
    filename="*q8_0.gguf",
    verbose=False,
    n_ctx=2048,
#     n_gpu_layers=-1
)


prompt = """
You are a helpful Doctor, Help the patient below with their queries and provide a detail ansewr to them
Patient: Why DOes my nose hurt so bad Doctor, its like there is severe suffocation and my thorat is also hurting, Can you devise a full Weekly prescription as to what i shoudl take and also tell me safety
precautions to take, please, i would be forever thankful?
Doctor:
"""


output = llm(
      prompt, # Prompt
      max_tokens=None, # Generate up to 32 tokens, set to None to generate up to the end of the context window
      stop=["Q:", "\n"],# Stop generating just before the model would generate a new question
      echo=True # Echo the prompt back in the output
)



print(output['choices'][0]['text'])


You are a helpful Doctor, Help the patient below with their queries and provide a detail ansewr to them
Patient: Why DOes my nose hurt so bad Doctor, its like there is severe suffocation and my thorat is also hurting, Can you devise a full Weekly prescription as to what i shoudl take and also tell me safety
precautions to take, please, i would be forever thankful?
Doctor: 
It sounds like you may have an allergic reaction to your nasal polystirin (also known as saline or antihistamine nasal spray). This is a common symptom of many different respiratory illnesses. As for the safety and precautions, I'd recommend that you avoid any potentially allergenic foods or drinks such as eggs, peanuts, and dairy products as well as certain medications and exposure to extreme temperatures and humidity.


In [ ]:
!pip install flask-ngrok

In [ ]:
from flask import Flask, jsonify, request
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

@app.route(“/”)
def hello():
    return “Hello World!! from Google Colab”

if __name__ == ‘__main__’:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: on


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1042, in launch_instance
    app.initialize(argv)
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 113, in inner
    return method(app, *args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 654, in initialize
    self.init_sockets()
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 331, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  File "/opt/conda/lib/python3.10/site-packages/

SystemExit: 1

In [ ]:
!ngrok authtoken "2SI4PeRpXIA9iTShpVcRDvliYf5_78pcZKUJSaCd5aGZKgGT1"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from llama_cpp import Llama

app = Flask(__name__)
run_with_ngrok(app)
# Load the model
checkpoint = "Qwen/Qwen2-0.5B-Instruct-GGUF"
filename = "*q8_0.gguf"
llm = Llama.from_pretrained(
    repo_id=checkpoint,
    filename=filename,
    verbose=False,
    n_ctx=2048,
#     n_gpu_layers=-1
)


@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    prompt = data['prompt']
    max_tokens = data.get('max_tokens', 32)

    # Generate text
    response = llm(prompt,
                   max_tokens=max_tokens,
                   stop=["Q:", "\n"],
                   echo=True)

    return jsonify(response)

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off
Usage of ngrok requires a verified account and authtoken.

Sign up for an account: https://dashboard.ngrok.com/signup
Install your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken

ERR_NGROK_4018



Exception in thread Thread-9:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/opt/conda/lib/python3.10/site-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/opt/conda/lib/python3.10/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py", line 790, in urlopen
    response = self._make_request(
  File "/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py", line 496, in _make_request
    conn.request(
  File "/opt/conda/lib/python3.10/site-packages/urllib3/connection.py", line 395, in request
    self.endheaders()
  File "/opt/conda/li

In [ ]:
from llama_cpp import Llama
from llama_cpp.llama_chat_format import MoondreamChatHandler

chat_handler = MoondreamChatHandler.from_pretrained(
  repo_id="vikhyatk/moondream2",
  filename="*mmproj*",
)

llm = Llama.from_pretrained(
  repo_id="vikhyatk/moondream2",
  filename="*text-model*",
  chat_handler=chat_handler,
  n_ctx=2048, # n_ctx should be increased to accommodate the image embedding
)

response = llm.create_chat_completion(
    messages = [
        {
            "role": "user",
            "content": [
                {"type" : "text", "text": "What's in this image?"},
                {"type": "image_url", "image_url": {"url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg" } }

            ]
        }
    ]
)
print(response["choices"][0]["text"])

In [ ]:
%%bash
# Install dependencies
# sudo apt-get update
# sudo apt-get install build-essential autoconf libtool pkg-config libssl-dev

# Download the source code
# git clone https://github.com/curl/curl.git

# Navigate to the source directory
# %cd '/kaggle/working/curl'

# Configure the build
# ./buildconf
# ./configure --with-ssl

# Compile the source code
# make

# Run tests (optional)
# make test

# Install the compiled software
# sudo make install

# Verify the installation
curl --version


In [ ]:
%%bash
curl --help

In [ ]:
%%bash
pwd